In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import regex as re
from lxml import etree as et
from itertools import repeat
import csv
import random
import time
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

In [ ]:
header_list = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML,                           like Gecko) Chrome/103.0.5060.66 Safari/537.36",
              "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0",
              "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393"]

base_url = "https://www.amazon.in/s?k="
smartwatch_brands = ['Apple', 'Noise', 'boAt', 'Honor', 'SAMSUNG', 'FITBIT', 'Amazfit', 'GARMIN', 'Huawei', 'FOSSIL', 'Dizo', 'Gizmore', 'Amazfit', 'Ambrane', 'Pebble', 'Zebronics', 'fireboltt']
#smartwatch_brands = ['Amazfit']
product_list = []
d = 0

In [ ]:
for brand in tqdm_notebook(smartwatch_brands):
  for page_no in range(3):
    page_url = base_url + brand + '-smart-watch-' '&page=' + str(page_no)
    user_agent = random.choice(header_list)
    header = {"User-Agent": user_agent}
    webpage = requests.get(page_url, headers = header)
    if webpage.status_code == 200:
      soup1 = BeautifulSoup(webpage.content, 'html.parser')
      soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
      divs = soup2.find_all('div', {'class' : 'sg-col-20-of-24 s-matching-dir sg-col-16-of-20 sg-col sg-col-8-of-12 sg-col-12-of-16'} )
      for div in divs:
        swlink = div.find_all('a', {'class' : 'a-link-normal s-no-outline'})
        for l in swlink:
          product_list.append([brand, 'https://www.amazon.in' + l.get('href')])
    else:
      d += 1

  0%|          | 0/17 [00:00<?, ?it/s]

In [ ]:
print(len(product_list), d)

975 0


In [ ]:
#[[Create df]]
cols = ['Url', 'Brand', 'Current Price', 'Original Price', 'Discount Percentage', 
        'Number of Models', 'Rating', 'Number of Ratings', 'Picture', 'Highlights',
        'Reviews']
feature_list = ['Brand', 'Manufacturer', 'Model', 'Model Name', 'Model Year',
                  'Product Dimensions', 'Batteries', 'Operating System', 'Item Model Number'
                  'Date First Available', 'Importer',
                'Compatible Devices', 'Special Features', 'Mounting Hardware',
                'Number Of Items', 'Battery Cell Composition', 'ASIN',
                'Standing screen display size', 'Wattage', 'Battery Average Life',
                'Wireless Type', 'Device interface - primary', 'Material',
                'Country of Origin', 'Item Weight']

cols.extend(feature_list)
amaz = pd.DataFrame(columns = cols)

In [ ]:
#[[feature_result_transform]]
def feature_result_transform(feature_list, feature_result_dict):
  
  result_list = []
  result_list.extend(repeat(np.nan, len(feature_list)))

  for key in feature_result_dict.keys():
    for i in range(len(feature_list)):
      if feature_list[i] == key:
        result_list[i] = feature_result_dict[key]   
    
  return result_list

In [ ]:
for brand,link in tqdm_notebook(product_list):
  url = link
  user_agent = random.choice(header_list)
  header = {"User-Agent": user_agent}
  webpage = requests.get(url, headers = header)
  if webpage.status_code == 200:
    soup1 = BeautifulSoup(webpage.content, 'html.parser')
    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

    brand = brand.lower()
    prices = soup2.find(id = 'apex_desktop')
    try:
      prices = prices.find_all('span', {'class' : 'a-offscreen'})
      cur_price = int(re.sub('₹|,', '', prices[0].get_text().strip()))
      org_price = int(re.sub('₹|,', '', prices[1].get_text().strip()))
      discount = ((org_price - cur_price)/org_price) * 100  
    except:
      cur_price = np.nan
      org_price = np.nan
      discount = np.nan
    #print(cur_price, org_price, link)

    try:
      nmodels = soup2.find(id = 'variation_color_name')
      nmodels = len(nmodels.find_all('li'))
    except:
      nmodels = np.nan
    #print(nmodels)

    try:
      pic = soup2.find(id = 'landingImage').get('src')
    except:
      pic = np.nan
    #print(pic)

    try:
      highlights = soup2.find('ul', {'class' : 'a-unordered-list a-vertical a-spacing-mini'})
      highlights = highlights.find_all('span', {'class' : 'a-list-item'})
      highlights = [h.get_text().strip() for h in highlights]
    except:
      highlights = np.nan

    try:
      table = soup2.find(id = 'prodDetails')

      features = table.find_all('th', {'class': "a-color-secondary a-size-base prodDetSectionEntry"})
      features = [f.get_text().strip() for f in features]
      
      results = table.find_all('td', {'class' : 'a-size-base prodDetAttrValue'})
      results = [r.get_text().strip().replace('\u200e', '') for r in results]
      
      dicted = {}
      for i in range(len(features)):
        dicted[features[i]] = results[i]    
      #print(dicted)
      #results = feature_result_transform(dicted)
    except:
      #print('---')
      pass

    try:
      revs = soup2.find('div', {'class' : 'card-padding'})
      revs = revs.find_all('div', {'class' : 'a-row a-spacing-small review-data'})
      reviews = [rev.get_text().strip().replace('\n', '') for rev in revs]
    except:
      reviews = np.nan
    
    try:
      cust_revs_box = soup2.find('span', {'class' : 'cr-widget-TitleRatingsAndHistogram'})
      text_rating = cust_revs_box.find('span', {'class' : 'a-size-medium a-color-base'}).get_text().strip()
      nglobal_ratings = cust_revs_box.find('span', {'class' : 'a-size-base a-color-secondary'}).get_text().strip()
      nglobal_ratings = int(re.sub(',', '', nglobal_ratings.split()[0]))
    except:
      text_rating = np.nan
      nglobal_ratings = np.nan

    results = feature_result_transform(feature_list, dicted)
    row = [url, brand, cur_price, org_price, discount, nmodels, text_rating, nglobal_ratings, pic, highlights, reviews]
    row.extend(results)
    amaz.loc[len(amaz.index)] = row   


  0%|          | 0/975 [00:00<?, ?it/s]

In [ ]:
amaz.tail()

,Url,Brand,Current Price,Original Price,Discount Percentage,Number of Models,Rating,Number of Ratings,Picture,Highlights,...,Battery Cell Composition,ASIN,Standing screen display size,Wattage,Battery Average Life,Wireless Type,Device interface - primary,Material,Country of Origin,Item Weight
831,https://www.amazon.in/Fire-Boltt-Smartwatch-Bl...,fireboltt,1799.0,9999.0,82.008201,NaN,4.2 out of 5,36092.0,https://m.media-amazon.com/images/W/IMAGERENDE...,[【Bluetooth Calling Watch】- Fire-Boltt Talk 2 ...,...,NaN,B0BF54972T,1.83 Inches,NaN,3 Days,Bluetooth,"Touchscreen, Buttons",NaN,China,50 g
832,https://www.amazon.in/Fire-Boltt-Outdoor-Bluet...,fireboltt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,B0BF54972T,1.83 Inches,NaN,3 Days,Bluetooth,"Touchscreen, Buttons",NaN,China,50 g
833,https://www.amazon.in/Fire-Boltt-Display-Resis...,fireboltt,1799.0,8999.0,80.008890,NaN,4 out of 5,10068.0,https://m.media-amazon.com/images/I/41TUCxf4ik...,[Fire-Boltt is India' No 1 Wearable Watch Bran...,...,NaN,B094QXS8NJ,1.3 Inches,240 Watts,8 Days,Bluetooth,"Touchscreen, Buttons",Silicone,India,51 g
834,https://www.amazon.in/Fire-Boltt-Bluetooth-Sma...,fireboltt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,B094QXS8NJ,1.3 Inches,240 Watts,8 Days,Bluetooth,"Touchscreen, Buttons",Silicone,India,51 g
835,https://www.amazon.in/Fire-Boltt-Bluetooth-Cal...,fireboltt,2499.0,9999.0,75.007501,NaN,4 out of 5,5698.0,https://m.media-amazon.com/images/I/41o9SR3g0R...,[Fire-Boltt is India' No 1 Wearable Watch Bran...,...,NaN,B0BXSP5BYN,1.3 Inches,240 Watts,NaN,Bluetooth,"Touchscreen, Microphone, Buttons",NaN,China,36.8 g


In [ ]:
amaz.to_csv('amaz.csv')